In [9]:
import mysql.connector

def new_conn():
    return mysql.connector.connect(
        user='root',
        password='example',
        host='mysql',
        database='example'
    )

try:
    # Host is `mysql` because we are in Docker's network

    conn = new_conn()
    cursor = conn.cursor()
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))

In [10]:
CREATE_TABLE_EMPS_STMT = '''
CREATE TABLE emps (
    empno INT NOT NULL,
    empname VARCHAR(50),
    job VARCHAR(30),
    PRIMARY KEY (empno)
);
'''

try:
    cursor.execute(CREATE_TABLE_EMPS_STMT)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))

In [11]:
CREATE_TABLE_SALARY_STMT = '''
CREATE TABLE salary (
    empno INT NOT NULL,
    salary INT,
    PRIMARY KEY (empno)
);
'''

try:
    cursor.execute(CREATE_TABLE_SALARY_STMT)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))

In [12]:
# Link `salary` with `emps` table by defining a foreign key
ADD_FOREIGN_KEY_STMT = '''
ALTER TABLE salary
ADD FOREIGN KEY (empno)
REFERENCES emps (empno);
'''

try:
    cursor.execute(ADD_FOREIGN_KEY_STMT)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))

In [13]:
CREATE_TABLE_ORDERS_STMT = '''
CREATE TABLE orders (
    pono INT NOT NULL,
    empno INT NOT NULL,
    total INT,
    PRIMARY KEY (pono),
    FOREIGN KEY (empno) REFERENCES emps (empno)
);
'''

try:
    cursor.execute(CREATE_TABLE_ORDERS_STMT)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))

Proceed to insert records

In [14]:
try:
    cursor = conn.cursor()

    # Creates data of employees
    emps = [
        (9001, "Jeff Russell", "sales"),
        (9002, "Jane Boorman", "sales"),
        (9003, "Tom Heints", "sales")
    ]

    # Define insertion query
    query_add_emp = ('''INSERT INTO emps (empno, empname, job)
        VALUES (%s, %s, %s)''')

    # Execute the insertion query per employee. in production is
    # more efficient to execute a single transaction using a INSERT MANY
    # approach.
    for emp in emps:
        cursor.execute(query_add_emp, emp)

    salary = [
        (9001, 3000),
        (9002, 2800),
        (9003, 2500)
    ]

    query_add_salary = ('''INSERT INTO salary (empno, salary)
        VALUES (%s, %s)''')

    for sal in salary:
        cursor.execute(query_add_salary, sal)

    orders = [
        (2608, 9001, 35),
        (2617, 9001, 35),
        (2620, 9001, 139),
        (2621, 9002, 95),
        (2626, 9002, 218),
    ]

    query_add_order = ('''INSERT INTO orders (pono, empno, total)
        VALUES (%s, %s, %s)''')

    for order in orders:
        cursor.execute(query_add_order, order)

    conn.commit()
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

With our data loaded into the database, we can now fetch tables and even join tables on queries for richer data results

In [16]:
try:
    conn = new_conn()
    cursor = conn.cursor()
    query = ('SELECT * FROM emps WHERE empno > %s')
    cursor.execute(query, (9000,))

    for (empno, empname, job) in cursor:
        print(f'{empno} {empname}, {job}')
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

9001 Jeff Russell, sales
9002 Jane Boorman, sales
9003 Tom Heints, sales


Joining two tables, employees and salaries

In [20]:
try:
    conn = new_conn()
    cursor = conn.cursor()
    query = ('''
    SELECT
        e.empno,
        e.empname,
        e.job,
        s.salary
    FROM emps e
    JOIN salary s ON e.empno = s.empno
    WHERE e.empno > %s
    ''')
    cursor.execute(query, (9000,))

    for (empno, empname, job, salary) in cursor:
        print(f'{empno}\t{empname}\t{job}\t{salary}')
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

9001	Jeff Russell	sales	3000
9002	Jane Boorman	sales	2800
9003	Tom Heints	sales	2500


Now we perform a One to Many Join to include each employee orders.

In [25]:
try:
    conn = new_conn()
    cursor = conn.cursor()
    query = ('''
    SELECT
        e.empno,
        e.empname,
        e.job,
        s.salary
    FROM emps e
    JOIN salary s ON e.empno = s.empno
    LEFT JOIN orders o ON e.empno = o.empno
    WHERE e.empno > %s
    ''')
    cursor.execute(query, (9000,))

    for (empno, empname, job, salary) in cursor:
        print(f'{empno}\t{empname}\t{job}\t{salary}')
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

9001	Jeff Russell	sales	3000
9001	Jeff Russell	sales	3000
9001	Jeff Russell	sales	3000
9002	Jane Boorman	sales	2800
9002	Jane Boorman	sales	2800
9003	Tom Heints	sales	2500


Now we fetch data from Yahoo Finance using the `yfinance` library and then such data is inserted into the MySQL database.
For simplicity the same database is used.

Previous tables are dropped to keep current DB tidy.

In [26]:
try:
    conn = new_conn()
    cursor = conn.cursor()
    query = ('''
    DROP TABLE orders;
    DROP TABLE salary;
    DROP TABLE emps;
    ''')
    cursor.execute(query)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

Now stocks tables are created.

In [51]:
query = '''
CREATE TABLE stocks (
    ticker VARCHAR(10),
    date VARCHAR(10),
    price DECIMAL(15,2)
);
'''

try:
    conn = new_conn()
    cursor = conn.cursor()
    cursor.execute(query)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

In [52]:
import yfinance as yf

FIVE_DAYS_PERIOD = '5d'

data = []
tickers = ['TSLA', 'META', 'ORCL', 'AMZN']

for ticker in tickers:
    tkr = yf.Ticker(ticker)
    hist = tkr.history(period=FIVE_DAYS_PERIOD).reset_index()
    records = hist[['Date', 'Close']].to_records(index=False)
    records = list(records)
    records = [(ticker, str(elem[0])[:10], elem[1]) for elem in records]
    data = data + records

display(data)

[('TSLA', '2024-04-24', 162.1300048828125),
 ('TSLA', '2024-04-25', 170.17999267578125),
 ('TSLA', '2024-04-26', 168.2899932861328),
 ('TSLA', '2024-04-29', 194.0500030517578),
 ('TSLA', '2024-04-30', 183.27999877929688),
 ('META', '2024-04-24', 493.5),
 ('META', '2024-04-25', 441.3800048828125),
 ('META', '2024-04-26', 443.2900085449219),
 ('META', '2024-04-29', 432.6199951171875),
 ('META', '2024-04-30', 430.1700134277344),
 ('ORCL', '2024-04-24', 115.33999633789062),
 ('ORCL', '2024-04-25', 114.88999938964844),
 ('ORCL', '2024-04-26', 117.20999908447266),
 ('ORCL', '2024-04-29', 116.48999786376953),
 ('ORCL', '2024-04-30', 113.75),
 ('AMZN', '2024-04-24', 176.58999633789062),
 ('AMZN', '2024-04-25', 173.6699981689453),
 ('AMZN', '2024-04-26', 179.6199951171875),
 ('AMZN', '2024-04-29', 180.9600067138672),
 ('AMZN', '2024-04-30', 175.0)]

In [53]:
query = '''
INSERT INTO stocks (
    ticker,
    date,
    price
) VALUES (
    %s,
    %s,
    %s
)
'''

try:
    conn = new_conn()
    cursor = conn.cursor()
    cursor.executemany(query, data)
    conn.commit()
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

In [56]:
query = '''
SELECT ticker, date, price FROM stocks
'''

try:
    conn = new_conn()
    cursor = conn.cursor()
    cursor.execute(query)

    for (ticker, date, price) in cursor:
        print(f'{ticker}\t{date}\t{price}')
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

TSLA	2024-04-24	162.13
TSLA	2024-04-25	170.18
TSLA	2024-04-26	168.29
TSLA	2024-04-29	194.05
TSLA	2024-04-30	183.28
META	2024-04-24	493.50
META	2024-04-25	441.38
META	2024-04-26	443.29
META	2024-04-29	432.62
META	2024-04-30	430.17
ORCL	2024-04-24	115.34
ORCL	2024-04-25	114.89
ORCL	2024-04-26	117.21
ORCL	2024-04-29	116.49
ORCL	2024-04-30	113.75
AMZN	2024-04-24	176.59
AMZN	2024-04-25	173.67
AMZN	2024-04-26	179.62
AMZN	2024-04-29	180.96
AMZN	2024-04-30	175.00


The `LAG` function in MySQL can be used when the `SELECT` instruction is in context, to access the previous row
data from the current row.

Using the `PARTITION BY` clause within the `OVER` clause, the dataset is divided into groups, in this scenario one per ticker.

In [62]:
query = '''
SELECT
	date,
	ticker,
	price,
	LAG(price) OVER (PARTITION BY ticker ORDER BY date) AS prev_price
FROM
	stocks;
'''

try:
    conn = new_conn()
    cursor = conn.cursor()
    cursor.execute(query)

    for (date, ticker, price, prev_price) in cursor:
        print(f'{ticker}\t{date}\t{price}\t{prev_price}')
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

AMZN	2024-04-24	176.59	None
AMZN	2024-04-25	173.67	176.59
AMZN	2024-04-26	179.62	173.67
AMZN	2024-04-29	180.96	179.62
AMZN	2024-04-30	175.00	180.96
META	2024-04-24	493.50	None
META	2024-04-25	441.38	493.50
META	2024-04-26	443.29	441.38
META	2024-04-29	432.62	443.29
META	2024-04-30	430.17	432.62
ORCL	2024-04-24	115.34	None
ORCL	2024-04-25	114.89	115.34
ORCL	2024-04-26	117.21	114.89
ORCL	2024-04-29	116.49	117.21
ORCL	2024-04-30	113.75	116.49
TSLA	2024-04-24	162.13	None
TSLA	2024-04-25	170.18	162.13
TSLA	2024-04-26	168.29	170.18
TSLA	2024-04-29	194.05	168.29
TSLA	2024-04-30	183.28	194.05


In [67]:
import pandas as pd

query = '''
SELECT
	s.*
FROM
	stocks AS s
	LEFT JOIN ( SELECT DISTINCT
			(ticker)
		FROM (
			SELECT
				price / LAG(price) OVER (PARTITION BY ticker ORDER BY date) AS dif,
				ticker
			FROM
				stocks) AS b
		WHERE
			dif < 0.99) AS a ON a.ticker = s.ticker
WHERE
	a.ticker IS NULL
'''

try:
    conn = new_conn()
    cursor = conn.cursor()


    df_stocks = pd.read_sql(query, con=conn)
    df_stocks = df_stocks.set_index(['ticker', 'date'])
    display(df_stocks)
except mysql.connector.Error as err:
    print('MySQL Error')
    print('(Code: {}) {}'.format(err.errno, err.msg))
finally:
    cursor.close()
    conn.close()

/tmp/ipykernel_28/3778911844.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_stocks = pd.read_sql(query, con=conn)


,,price
ticker,date,
